In [40]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas import Series
import numpy as np

import matplotlib.pylab as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
import re
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm
import itertools

In [123]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
*The raw code for this Jupyter notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series
import numpy as np

import matplotlib.pylab as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [157]:
def get_all_predictions(dataframe):
    pred_df = pd.DataFrame(index = index_for_df)
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    pdqs = [(x[0], x[1], x[2], 1) for x in list(itertools.product(p, d, q))]
    df = dataframe.drop('5. volume', axis = 1)
    for metric in df.columns:
        ans = []
        for comb in pdq:
            for combs in pdqs:
                try:
                    mod = sm.tsa.statespace.SARIMAX(dataframe[metric],
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
                    output = mod.fit()
                    ans.append([comb, combs, output.aic])
                except:
                    continue
        ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
        ARIMA_MODEL = sm.tsa.statespace.SARIMAX(dataframe[metric],
                                order=ans_df.loc[ans_df['aic'].idxmin()][0],
                                seasonal_order=ans_df.loc[ans_df['aic'].idxmin()][1],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
        print(ans_df.loc[ans_df['aic'].idxmin()][0])
        print(ans_df.loc[ans_df['aic'].idxmin()][1])
        output = ARIMA_MODEL.fit()
        pred = output.get_prediction(start=pd.to_datetime('2018-08-31 15:45:00'), end=pd.to_datetime('2018-08-31 14:00:00'),dynamic=False)
        predictions = pd.DataFrame(pred.predicted_mean)
        pred_df[metric] = predictions
    return pred_df

In [83]:
index_for_df = apple.index[(apple.index < '2018-08-31 15:50:00')& (apple.index > '2018-08-31 13:55:00')]

In [2]:
import alpha_vantage
from alpha_vantage.timeseries import TimeSeries
stock = TimeSeries(key='1250F9WWA3Z77BIK')

In [3]:
nflx = pd.read_csv('./csv/nflx_daily_stock.csv', index_col='Unnamed: 0')
fb = pd.read_csv('./csv/fb_daily_stock.csv', index_col='Unnamed: 0')
apple = pd.read_csv('./csv/apple_daily_stock.csv', index_col='Unnamed: 0')
fb_prediction_df = pd.read_csv('./csv/fb_predictions.csv', index_col='Unnamed: 0')
apple_prediction_df = pd.read_csv('apple_predictions.csv', index_col='Unnamed: 0')
netflix_prediction_df = pd.read_csv('netflix_predictions.csv', index_col='Unnamed: 0')

# Candlestick Charts 8-31-2018

In [6]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime

# ts_close = pd.DataFrame(fb['4. close'])
# ts_close = ts_close['4. close'].astype(float)
# rolmean = ts_close.rolling(window=6).mean()
# orig_close = plt.plot(ts, color='blue',label='Original')
# mean_close = plt.plot(rolmean, color='red', label='Rolling Mean')
#     std = plt.plot(rolstd, color='black', label = 'Rolling Std')
# plt.legend(loc='best')
# plt.title('Rolling Mean & Standard Deviation')
# plt.show(block=False)


layout = {'xaxis': {'rangeslider':{'visible':False}},
    'title': 'Facebook Stock 08-31-2018',
    'yaxis': {'title': 'Stock Price'},'shapes': [{
        'x0': '2018-08-31 14:00:00', 'x1': '2018-08-31 14:00:00',
        'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
        'line': {'color': 'rgb(30,30,30)', 'width': 1}
    }],'annotations': [{
        'x': '2018-08-31 14:00:00', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
        'showarrow': False, 'xanchor': 'left',
        'text': 'Predictions Begin'
    }]}


trace = go.Candlestick(name = 'Price',x=nflx.index,
                       open=fb['1. open'],
                       high=fb['2. high'],
                       low=fb['3. low'],
                       close=fb['4. close'],
                       increasing=dict(line=dict(color= 'Blue')),
                       decreasing=dict(line=dict(color= 'Silver')))

trace2 = go.Candlestick(name = 'Predicted Price',x=fb_prediction_df.index,
                       open=fb_prediction_df.open_pred,
                       high=fb_prediction_df.high_pred,
                       low=fb_prediction_df.low_pred,
                       close=fb_prediction_df.close_pred,
                       increasing=dict(line=dict(color= 'Green')),
                       decreasing=dict(line=dict(color= 'Red')))

# trace2 = go.Scatter(name = 'Close Price MA',
#     x = rolmean.index,
#     y = rolmean.values, line = dict(
#         color = ('red'), o))

data = [trace, trace2]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='plot from API (15)')

In [130]:
join_fb = fb[(fb.index < '2018-08-31 15:50:00') & (fb.index > '2018-08-31 13:55:00')]
# join_fb.columns = ['actual_open', 'actual_high', 'actual_low', 'actual_close', 'actual_volume']
# fb_join = fb_prediction_df.join(join_fb)
# fb_join['close_difference'] = fb_join.close_pred - fb_join.actual_close
# fb_join['open_difference'] = fb_join.open_pred - fb_join.actual_open
# fb_join['high_difference'] = fb_join.high_pred - fb_join.actual_high
# fb_join['low_difference'] = fb_join.low_pred - fb_join.actual_low
# display(fb_join.close_difference.describe())
# display(fb_join.open_difference.describe())
# display(fb_join.high_difference.describe())
# display(fb_join.low_difference.describe())

In [131]:
join_fb.columns = ['actual_open', 'actual_high', 'actual_low', 'actual_close', 'actual_volume']
fb_join = fb_prediction_df.join(join_fb)
fb_join['close_difference'] = fb_join.close_pred - fb_join.actual_close
fb_join['open_difference'] = fb_join.open_pred - fb_join.actual_open
fb_join['high_difference'] = fb_join.high_pred - fb_join.actual_high
fb_join['low_difference'] = fb_join.low_pred - fb_join.actual_low

In [137]:
display(fb_join.close_difference.describe())
display(fb_join.open_difference.describe())
display(fb_join.high_difference.describe())
display(fb_join.low_difference.describe())

count    22.000000
mean      0.006818
std       0.138313
min      -0.235100
25%      -0.083675
50%       0.010000
75%       0.067500
max       0.340000
Name: close_difference, dtype: float64

count    22.000000
mean     -0.004545
std       0.131522
min      -0.220000
25%      -0.110000
50%       0.015000
75%       0.047425
max       0.300100
Name: open_difference, dtype: float64

count    22.000000
mean      0.012018
std       0.137539
min      -0.286967
25%      -0.051162
50%       0.009452
75%       0.082324
max       0.335000
Name: high_difference, dtype: float64

count    22.000000
mean      0.002415
std       0.131138
min      -0.249679
25%      -0.060606
50%      -0.004802
75%       0.081127
max       0.325198
Name: low_difference, dtype: float64

In [93]:
apple_prediction_df = get_all_predictions(apple)

In [98]:
apple_prediction_df.to_csv('apple_predictions.csv')

In [97]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime


layout = {'xaxis': {'rangeslider':{'visible':False}},
    'title': 'Apple Stock 08-31-2018',
    'yaxis': {'title': 'Stock Price'},'shapes': [{
        'x0': '2018-08-31 14:00:00', 'x1': '2018-08-31 14:00:00',
        'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
        'line': {'color': 'rgb(30,30,30)', 'width': 1}
    }],'annotations': [{
        'x': '2018-08-31 14:00:00', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
        'showarrow': False, 'xanchor': 'left',
        'text': 'Predictions Begin'
    }]}


trace = go.Candlestick(name = 'Actual Price',x=apple.index,
                       open=apple['1. open'],
                       high=apple['2. high'],
                       low=apple['3. low'],
                       close=apple['4. close'],
                       increasing=dict(line=dict(color= 'Black')),
                       decreasing=dict(line=dict(color= 'Silver')))

trace2 = go.Candlestick(name = 'Predicted Price',x=apple_prediction_df.index,
                       open=apple_prediction_df['1. open'],
                       high=apple_prediction_df['2. high'],
                       low=apple_prediction_df['3. low'],
                       close=apple_prediction_df['4. close'],
                       increasing=dict(line=dict(color= 'Green')),
                       decreasing=dict(line=dict(color= 'Red')))


# trace2 = go.Scatter(name = 'Close Price MA',
#     x = rolmean.index,
#     y = rolmean.values, line = dict(
#         color = ('red'), o))

data = [trace,trace2]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='plot from API (16)')

In [138]:
join_apple = apple[(apple.index < '2018-08-31 15:50:00') & (apple.index > '2018-08-31 13:55:00')]

In [143]:
join_apple.columns = ['actual_open', 'actual_high', 'actual_low', 'actual_close', 'actual_volume']
apple_join = apple_prediction_df.join(join_apple)
apple_join['close_difference'] = apple_join['4. close'] - apple_join.actual_close
apple_join['open_difference'] = apple_join['1. open'] - apple_join.actual_open
apple_join['high_difference'] = apple_join['2. high'] - apple_join.actual_high
apple_join['low_difference'] = apple_join['3. low'] - apple_join.actual_low

In [144]:
display(apple_join.close_difference.describe())
display(apple_join.open_difference.describe())
display(apple_join.high_difference.describe())
display(apple_join.low_difference.describe())

count    22.000000
mean      0.013465
std       0.194486
min      -0.275567
25%      -0.116620
50%      -0.031214
75%       0.068373
max       0.475825
Name: close_difference, dtype: float64

count    22.000000
mean      0.002605
std       0.211075
min      -0.326030
25%      -0.129948
50%      -0.029738
75%       0.072788
max       0.492722
Name: open_difference, dtype: float64

count    22.000000
mean      0.004091
std       0.158138
min      -0.300100
25%      -0.092425
50%       0.004950
75%       0.095075
max       0.300000
Name: high_difference, dtype: float64

count    22.000000
mean     -0.004414
std       0.167489
min      -0.335100
25%      -0.136276
50%       0.004879
75%       0.110213
max       0.260603
Name: low_difference, dtype: float64

In [118]:
netflix_prediction_df = get_all_predictions(nflx)

In [121]:
netflix_prediction_df.to_csv('netflix_predictions.csv')

In [120]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime

layout = {'xaxis': {'rangeslider':{'visible':False}},
    'title': 'Netflix Stock 08-31-2018',
    'yaxis': {'title': 'Stock Price'},'shapes': [{
        'x0': '2018-08-31 14:00:00', 'x1': '2018-08-31 14:00:00',
        'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
        'line': {'color': 'rgb(30,30,30)', 'width': 1}
    }],'annotations': [{
        'x': '2018-08-31 14:00:00', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
        'showarrow': False, 'xanchor': 'left',
        'text': 'Predictions Begin'
    }]}

trace = go.Candlestick(name = 'Actual Price',x=nflx.index,
                       open=nflx['1. open'],
                       high=nflx['2. high'],
                       low=nflx['3. low'],
                       close=nflx['4. close'],
                       increasing=dict(line=dict(color= 'Red')),
                       decreasing=dict(line=dict(color= 'Black')))

trace2 = go.Candlestick(name = 'Predicted Price',x=netflix_prediction_df.index,
                       open=netflix_prediction_df['1. open'],
                       high=netflix_prediction_df['2. high'],
                       low=netflix_prediction_df['3. low'],
                       close=netflix_prediction_df['4. close'],
                       increasing=dict(line=dict(color= 'Green')),
                       decreasing=dict(line=dict(color= 'Silver')))

data = [trace, trace2]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='plot from API (18)')

In [153]:
# nflx = nflx.drop('interval',axis =1)
join_nflx = nflx[(nflx.index < '2018-08-31 15:50:00') & (nflx.index > '2018-08-31 13:55:00')]
join_nflx.columns = ['actual_open', 'actual_high', 'actual_low', 'actual_close', 'actual_volume']
nflx_join = netflix_prediction_df.join(join_nflx)
nflx_join['close_difference'] = nflx_join['4. close'] - nflx_join.actual_close
nflx_join['open_difference'] = nflx_join['1. open'] - nflx_join.actual_open
nflx_join['high_difference'] = nflx_join['2. high'] - nflx_join.actual_high
nflx_join['low_difference'] = nflx_join['3. low'] - nflx_join.actual_low

In [155]:
display(nflx_join.close_difference.describe())
display(nflx_join.open_difference.describe())
display(nflx_join.high_difference.describe())
display(nflx_join.low_difference.describe())

count    22.000000
mean     -0.217631
std       0.459140
min      -1.576706
25%      -0.434183
50%      -0.173279
75%       0.015096
max       0.454100
Name: close_difference, dtype: float64

count    22.000000
mean     -0.193638
std       0.478764
min      -1.531282
25%      -0.390712
50%      -0.121451
75%       0.151308
max       0.447062
Name: open_difference, dtype: float64

count    22.000000
mean     -0.097944
std       0.405771
min      -1.366914
25%      -0.269099
50%      -0.119491
75%       0.173028
max       0.726956
Name: high_difference, dtype: float64

count    22.000000
mean     -0.171364
std       0.357031
min      -0.943200
25%      -0.246875
50%      -0.140000
75%       0.070000
max       0.378000
Name: low_difference, dtype: float64